In [1]:
### FIRE DOMAIN ONLY
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import cPickle
from collections import defaultdict
import sys, os, math, re
os.environ['KERAS_BACKEND']='tensorflow'
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Dense
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
import matplotlib

from keras.models import model_from_json
from pandas import HDFStore 

Using TensorFlow backend.


In [9]:
    max_sen_len = 100
#     dir_list = ['./Data_100_2hyp/']
    dir_list = ['./Meeting_Data_100_9hyp/']
    input_flist = [ 'Inputs/meeting_2.npy', 'Inputs/meeting_5.npy', 'Inputs/meeting_10.npy', 'Inputs/meeting_20.npy', 'Inputs/meeting_50.npy']
    #input_flist = [ 'Inputs/fire_2.npy', 'Inputs/fire_5.npy', 'Inputs/fire_10.npy', 'Inputs/fire_20.npy', 'Inputs/fire_50.npy']
    output_flist = [ 'Outputs/res_meet2.npy', 'Outputs/res_meet5.npy', 'Outputs/res_meet10.npy', 'Outputs/res_meet20.npy', 'Outputs/res_meet50.npy']
    #output_flist = [ 'Outputs/res_fire2.npy', 'Outputs/res_fire5.npy', 'Outputs/res_fire10.npy', 'Outputs/res_fire20.npy', 'Outputs/res_fire50.npy']
    preds_flist = [ 'Preds/pred_meet2.npy', 'Preds/pred_meet5.npy', 'Preds/pred_meet10.npy', 'Preds/pred_meet20.npy', 'Preds/pred_meet50.npy']
    #preds_flist = [ 'Preds/pred_fire2.npy', 'Preds/pred_fire5.npy', 'Preds/pred_fire10.npy', 'Preds/pred_fire20.npy', 'Preds/pred_fire50.npy']
    weights_flist = [ 'Weights/weig_meet2.h5', 'Weights/weig_meet5.h5', 'Weights/weig_meet10.h5', 'Weights/weig_meet20.h5', 'Weights/weig_meet50.h5' ]
    #weights_flist = [ 'Weights/weig_fire2.h5', 'Weights/weig_fire5.h5', 'Weights/weig_fire10.h5', 'Weights/weig_fire20.h5', 'Weights/weig_fire50.h5' ]
    for dir_name in dir_list:

        for index_ in range(0,5):

            input_fname = input_flist[index_]
            output_fname = output_flist[index_]
            preds_fname = preds_flist[index_]
            weight_fname = weights_flist[index_]

            print (str (dir_name + input_fname))
            dataset = np.load(str(dir_name)+input_fname)
#             print(dataset.shape)
            dataset = dataset.astype('float32')

            # x=[]
            # for i in range(0,100):

            # 	x.append(i)
            # plt.figure('data')
            # for i in dataset:
            # 	plt.plot(x,i)
            # plt.show()

            np.random.seed(7)
            # np.random.shuffle(dataset)
            dataset = np.reshape(dataset, (dataset.shape[0], dataset.shape[1], 1))

            #################################
            # split into train and test sets
            val_split = 0.3
            train_size = int(len(dataset) * (1.0 - val_split))
            test_size = len(dataset) - train_size

            train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

            ### Due to 101 values being present
            x_train = train[:,:-1]
            y_train = train[:,:-1]
            print(y_train.shape)
            x_test = test[:,:-1]
            y_test = test[:,:-1]
            #################################

            #x, y = dataset[:,:-1], dataset[:,1:]

            ############### change model from sequence prediction to last value prediction ###################

            y_train_last = [[y_train[i,-1] for j in range(0,y_train.shape[1])] for i in range(0,y_train.shape[0])]

            y_test_last = [[y_test[i,-1] for j in range(0,y_test.shape[1])] for i in range(0,y_test.shape[0])]

            y_train_last = np.array(y_train_last)
            y_test_last = np.array(y_test_last)
            model = Sequential()
            model.add (LSTM (1, input_shape = (99,1), return_sequences=True, stateful=False, dtype='float32', dropout=0.3, recurrent_dropout=0.3))
            model.compile(loss="mse", optimizer="adam", metrics=['accuracy'])
            # model.fit (x_train, y_train, batch_size=1, epochs=20, validation_data=(x_test, y_test), shuffle=True)
            model.fit (x_train, y_train_last, batch_size=1, epochs=10, validation_data=(x_test, y_test_last), shuffle=True)
            predictions = model.predict(x_test)

            with open(str(dir_name + preds_fname),'wb') as pr: 
            	np.save(pr,predictions)

            with open(str(dir_name + output_fname),'wb') as res: 
            	np.save(res,y_test_last)

            model.save_weights(str(dir_name + weight_fname))

            # # load json and create model
            # json_file = open('model_boul10.json', 'r')
            # loaded_model_json = json_file.read()
            # json_file.close()
            # loaded_model = model_from_json(loaded_model_json)
            # # load weights into new model
            # loaded_model.load_weights("model_boul10.h5")
            # print("Loaded model from disk")

            # # evaluate loaded model on test data
            # loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
            # score = loaded_model.evaluate(X, Y, verbose=0)
            # print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

    with open('model_architecture.json', 'w') as f:
        f.write(model.to_json())

            ###################################################################################################

./Meeting_Data_100_9hyp/Inputs/meeting_2.npy
(700, 99, 1)
Train on 700 samples, validate on 300 samples
Epoch 1/10
700/700 [==============================] - 20s 29ms/step - loss: 0.1540 - accuracy: 0.0000e+00 - val_loss: 0.0968 - val_accuracy: 0.0000e+00
Epoch 2/10
700/700 [==============================] - 20s 28ms/step - loss: 0.1137 - accuracy: 0.0000e+00 - val_loss: 0.0927 - val_accuracy: 0.0000e+00
Epoch 3/10
700/700 [==============================] - 21s 30ms/step - loss: 0.1062 - accuracy: 0.0000e+00 - val_loss: 0.0914 - val_accuracy: 0.0000e+00
Epoch 4/10
700/700 [==============================] - 22s 31ms/step - loss: 0.1045 - accuracy: 0.0000e+00 - val_loss: 0.0911 - val_accuracy: 0.0000e+00
Epoch 5/10
700/700 [==============================] - 21s 31ms/step - loss: 0.1000 - accuracy: 0.0000e+00 - val_loss: 0.0902 - val_accuracy: 0.0000e+00
Epoch 6/10
700/700 [==============================] - 22s 31ms/step - loss: 0.0988 - accuracy: 0.0000e+00 - val_loss: 0.0901 - val_accur

In [10]:
for dir_name in dir_list:

        for index_ in range(0,5):
            print (str (dir_name + input_fname))
            input_fname = input_flist[index_]
            output_fname = output_flist[index_]
            preds_fname = preds_flist[index_]
            weight_fname = weights_flist[index_]
            data = str(dir_name)+output_fname
#             data = np.load(data)
#             print(data.shape)
            my = str(dir_name)+preds_fname
            my = np.load(my)
            print(my)

./Meeting_Data_100_9hyp/Inputs/meeting_50.npy
[[[0.17065273]
  [0.2958122 ]
  [0.37753847]
  ...
  [0.497775  ]
  [0.5011934 ]
  [0.5069751 ]]

 [[0.1688968 ]
  [0.29427382]
  [0.37300432]
  ...
  [0.49888614]
  [0.49790555]
  [0.5035142 ]]

 [[0.1688968 ]
  [0.28948736]
  [0.36775655]
  ...
  [0.51492083]
  [0.51602966]
  [0.5119812 ]]

 ...

 [[0.17172162]
  [0.29701412]
  [0.3775563 ]
  ...
  [0.50672644]
  [0.50336903]
  [0.50916076]]

 [[0.17179796]
  [0.29722658]
  [0.37646568]
  ...
  [0.5007275 ]
  [0.49915504]
  [0.5043759 ]]

 [[0.17145626]
  [0.29375926]
  [0.37405846]
  ...
  [0.50729513]
  [0.5089101 ]
  [0.5054891 ]]]
./Meeting_Data_100_9hyp/Inputs/meeting_2.npy
[[[0.267207  ]
  [0.40866828]
  [0.46241048]
  ...
  [0.5100216 ]
  [0.5011566 ]
  [0.4996047 ]]

 [[0.20549165]
  [0.32567194]
  [0.41601002]
  ...
  [0.508038  ]
  [0.5066205 ]
  [0.5056433 ]]

 [[0.23995046]
  [0.3665277 ]
  [0.43939355]
  ...
  [0.50187665]
  [0.50465643]
  [0.50326955]]

 ...

 [[0.2887483 ]
